In [2]:
from IPython.display import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from scipy import io
from sklearn import metrics 

In [3]:
def matRead(path):
    mat_file = io.loadmat(path)
    signal = mat_file['signal'][:,[0,2]]
    time = mat_file['time'][0]
    signal_ch1 = signal[:,0]
    signal_ch3 = signal[:,1]
    return time, signal_ch1, signal_ch3

In [4]:
test = matRead('data/1mm_Water_13Bar_[110-130].mat')

In [5]:
def clsfSig(rawdata):
    time = rawdata[0]
    tmp1 = rawdata[1]
    tmp3 = rawdata[2]
    
    sig1 = np.stack((time,tmp1),axis=1)
    sig3 = np.stack((time,tmp3),axis=1)
    
    sig1_good = sig1[sig1[:,0]<120]
    sig1_bad = sig1[sig1[:,0]>=120]
    
    sig3_good = sig3[sig3[:,0]<120]
    sig3_bad = sig3[sig3[:,0]>=120]
    
    tmp1_bad = sig1_bad[(sig1_bad[:,1] > np.max(sig1_good[:,1])) | (sig1_bad[:,1] < np.min(sig1_good[:,1]))]
    tmp3_bad = sig3_bad[(sig3_bad[:,1] > np.max(sig3_good[:,1])) | (sig3_bad[:,1] < np.min(sig3_good[:,1]))]
    
    good1 = np.stack((np.full(len(tmp1_bad),0),sig1_good[:,1][np.arange(len(tmp1_bad))]),axis=1)
    bad1 = np.stack((np.full(len(tmp1_bad),0),tmp1_bad[:,1]),axis=1)
    
    good3 = np.stack((np.full(len(tmp3_bad),0),sig3_good[:,1][np.arange(len(tmp3_bad))]),axis=1)
    bad3 = np.stack((np.full(len(tmp3_bad),0),tmp3_bad[:,1]),axis=1)
    return good1,bad1,good3,bad3

In [7]:
#AE SET 0.3



good_x = clsfSig(test)[0][:,0]
good_y = clsfSig(test)[0][:,1]

bad_x = clsfSig(test)[1][:,0]
bad_y = clsfSig(test)[1][:,1]

d_data = np.column_stack((good_x, good_y))
d_label = np.zeros(len(d_data))
s_data = np.column_stack((bad_x, bad_y))
s_label = np.ones(len(s_data))

newdata = [[0, 0.0027]]

dogs = np.concatenate((d_data, s_data))
labels = np.concatenate((d_label, s_label))

classes = {0:'정상신호', 1:'비정상신호'}

k = 3
knn = KNeighborsClassifier(n_neighbors = k) 
knn.fit(dogs, labels)
y_pred = knn.predict(newdata)
print('데이터', newdata, ', 판정 결과:', classes[y_pred[0]])

데이터 [[0, 0.0027]] , 판정 결과: 정상신호
